In [1]:
import sys,os
import numpy as np
import pandas as pd

from utils.method import read_bic_table

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# read sample annotations 
cohort = "BRCA"
annot = pd.read_csv("data/PanCan/"+cohort+".annotation.tsv.gz",sep ="\t",index_col=0)

annot["stage_3+4"] = annot["stage_3"] + annot["stage_4"] 
annot.loc[:,["stage_2","stage_3","stage_4","stage_3+4","age","sex"]].sum()

stage_2        622.0
stage_3        250.0
stage_4         20.0
stage_3+4      270.0
age          64019.0
sex           1086.0
dtype: float64

In [3]:
# read biclusters 
fname = cohort+".consensus_kmeans.42.bin=kmeans,pval=0.01,clust=WGCNA,ds=0,dch=0.995.biclsuters.tsv"
folder = "/home/olya/Downloads/"+cohort+"/"

biclusters = read_bic_table(folder+fname)
print("All biclusters:", biclusters.shape[0])
biclusters = biclusters.loc[biclusters["detected_n_times"]>1,:]
print("Detected 2+ times:", biclusters.shape[0])

All biclusters: 149
Detected 2+ times: 110


# Survival analysis

In [4]:
from utils.eval import  add_survival

covariates =["age","stage_2","stage_3","stage_4"]

In [5]:
# add survival analysis results to bicluster table
biclusters = add_survival(biclusters, annot, event= "OS",
                          surv_time ="OS.time",covariates = covariates)
biclusters = add_survival(biclusters, annot, event= "DFI",
                          surv_time ="DFI.time",covariates = covariates)
biclusters = add_survival(biclusters, annot, event= "PFI",
                          surv_time ="PFI.time",covariates = covariates)

biclusters

perfect separation for biclsuter of  12/1061 samples variances: 0.00 0.01
stage_4 with zero variance excluded
stage_4 with zero variance excluded
stage_4 with zero variance excluded
stage_4 with zero variance excluded
perfect separation for biclsuter of  12/923 samples variances: 0.00 0.01
stage_4 with zero variance excluded
stage_4 with zero variance excluded
stage_4 with zero variance excluded
stage_4 with zero variance excluded
stage_4 with zero variance excluded
stage_4 with zero variance excluded
stage_4 with zero variance excluded
stage_4 with zero variance excluded
stage_4 with zero variance excluded
stage_4 with zero variance excluded
stage_4 with zero variance excluded
stage_4 with zero variance excluded
stage_4 with zero variance excluded
stage_4 with zero variance excluded
stage_4 with zero variance excluded
stage_4 with zero variance excluded
stage_4 with zero variance excluded
stage_4 with zero variance excluded
stage_4 with zero variance excluded
stage_4 with zero varianc

/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/utils/__init__.py:1099: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead.
  problem_columns = (censors_only | deaths_only).difference(total).tolist()
perfect separation for biclsuter of  12/1061 samples variances: 0.00 0.01
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/utils/__init__.py:1099: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead.
  problem_columns = (censors_only | deaths_only).difference(total).tolist()
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1647: RuntimeWarning: overflow encountered in exp
  scores = weights * exp(dot(X, beta))
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1

/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1647: RuntimeWarning: overflow encountered in exp
  scores = weights * exp(dot(X, beta))
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1668: RuntimeWarning: invalid value encountered in add
  risk_phi_x = risk_phi_x + phi_x_i
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1669: RuntimeWarning: invalid value encountered in add
  risk_phi_x_x = risk_phi_x_x + phi_x_x_i
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1701: RuntimeWarning: invalid value encountered in multiply
  a1 = risk_phi_x_x * denom
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1703: RuntimeWarning: invalid value encountered in multiply
  summand = numer * denom[:, None]
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1708: RuntimeWarning: divide by zero encou

/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1647: RuntimeWarning: overflow encountered in exp
  scores = weights * exp(dot(X, beta))
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1668: RuntimeWarning: invalid value encountered in add
  risk_phi_x = risk_phi_x + phi_x_i
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1669: RuntimeWarning: invalid value encountered in add
  risk_phi_x_x = risk_phi_x_x + phi_x_x_i
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1701: RuntimeWarning: invalid value encountered in multiply
  a1 = risk_phi_x_x * denom
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1703: RuntimeWarning: invalid value encountered in multiply
  summand = numer * denom[:, None]
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1708: RuntimeWarning: divide by zero encou

/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1647: RuntimeWarning: overflow encountered in exp
  scores = weights * exp(dot(X, beta))
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1668: RuntimeWarning: invalid value encountered in add
  risk_phi_x = risk_phi_x + phi_x_i
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1669: RuntimeWarning: invalid value encountered in add
  risk_phi_x_x = risk_phi_x_x + phi_x_x_i
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1701: RuntimeWarning: invalid value encountered in multiply
  a1 = risk_phi_x_x * denom
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1703: RuntimeWarning: invalid value encountered in multiply
  summand = numer * denom[:, None]
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1708: RuntimeWarning: divide by zero encou

...

/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/utils/__init__.py:1099: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead.
  problem_columns = (censors_only | deaths_only).difference(total).tolist()
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1647: RuntimeWarning: overflow encountered in exp
  scores = weights * exp(dot(X, beta))
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1701: RuntimeWarning: invalid value encountered in multiply
  a1 = risk_phi_x_x * denom
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1703: RuntimeWarning: invalid value encountered in multiply
  summand = numer * denom[:, None]
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1708: RuntimeWarning: divide by zero encountered in log
  log

/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1647: RuntimeWarning: overflow encountered in exp
  scores = weights * exp(dot(X, beta))
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1701: RuntimeWarning: invalid value encountered in multiply
  a1 = risk_phi_x_x * denom
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1703: RuntimeWarning: invalid value encountered in multiply
  summand = numer * denom[:, None]
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1708: RuntimeWarning: divide by zero encountered in log
  log_lik = log_lik + dot(x_death_sum, beta) + weighted_average * log(denom).sum()
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1668: RuntimeWarning: invalid value encountered in add
  risk_phi_x = risk_phi_x + phi_x_i
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:

/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/utils/__init__.py:1099: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead.
  problem_columns = (censors_only | deaths_only).difference(total).tolist()
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1647: RuntimeWarning: overflow encountered in exp
  scores = weights * exp(dot(X, beta))
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1668: RuntimeWarning: invalid value encountered in add
  risk_phi_x = risk_phi_x + phi_x_i
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1669: RuntimeWarning: invalid value encountered in add
  risk_phi_x_x = risk_phi_x_x + phi_x_x_i
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1701: RuntimeWarning: invalid value encountered in multi

/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/utils/__init__.py:1099: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead.
  problem_columns = (censors_only | deaths_only).difference(total).tolist()
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1647: RuntimeWarning: overflow encountered in exp
  scores = weights * exp(dot(X, beta))
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1701: RuntimeWarning: invalid value encountered in multiply
  a1 = risk_phi_x_x * denom
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1703: RuntimeWarning: invalid value encountered in multiply
  summand = numer * denom[:, None]
/home/olya/anaconda3/lib/python3.8/site-packages/lifelines/fitters/coxph_fitter.py:1708: RuntimeWarning: divide by zero encountered in log
  log

,SNR,n_genes,n_samples,genes,samples,genes_up,genes_down,gene_indexes,sample_indexes,direction,...,DFI.LogR_p_value,DFI.p_value_BH,DFI.LogR_p_value_BH,PFI.p_value,PFI.HR,PFI.upper_95CI,PFI.lower_95CI,PFI.LogR_p_value,PFI.p_value_BH,PFI.LogR_p_value_BH
0,6.603591,2067,13,"{AC093381.2, CTA-992D9.10, OR51B3P, RP11-683O4...","{TCGA-A7-A26I-01, TCGA-AC-A2QH-01, TCGA-AC-A3Q...","{AC093381.2, CTA-992D9.10, OR51B3P, RP11-683O4...",{},"{24580, 24581, 32780, 8208, 24593, 24592, 2460...","{448, 810, 1035, 460, 45, 338, 506, 917, 182, ...",UP,...,0.526908,0.725944,0.789994,0.160283,2.742771,11.216134,0.670712,0.275693,0.220389,0.583198
1,4.498101,7,7,"{LINC00302, LCE1F, LCE1A, LCE2C, LCE2B, LCE1D,...","{TCGA-AC-A2QJ-01, TCGA-A2-A3XW-01, TCGA-E9-A29...","{LINC00302, LCE1F, LCE1A, LCE2C, LCE2B, LCE1D,...",{},"{15168, 15169, 15170, 15410, 15161, 15164, 15166}","{931, 518, 810, 338, 889, 634, 188}",UP,...,0.479807,0.618800,0.789994,NaN,NaN,NaN,NaN,0.757559,NaN,0.929303
2,4.037331,2,122,"{ACTG1P11, ACTG1P2}","{TCGA-AR-A24X-01, TCGA-A1-A0SI-01, TCGA-AR-A1A...","{ACTG1P11, ACTG1P2}",{},"{1905, 1897}","{514, 515, 4, 7, 1032, 522, 12, 1039, 16, 1045...",UP,...,0.083833,0.358917,0.431497,NaN,NaN,NaN,NaN,0.036803,NaN,0.238135
6,2.816966,4,13,"{TSPY3, TSPY10, TSPY8, TSPY2}","{TCGA-3C-AAAU-01, TCGA-A7-A6VY-01, TCGA-BH-A0D...","{TSPY3, TSPY10, TSPY8, TSPY2}",{},"{37048, 37044, 37052, 37046}","{32, 416, 451, 548, 1033, 842, 555, 207, 752, ...",UP,...,0.335689,NaN,0.659389,NaN,NaN,NaN,NaN,0.242414,NaN,0.554822
7,2.766905,5,75,"{PRR20B, PRR20A, PRR20D, PRR20E, PRR20C}","{TCGA-EW-A1PD-01, TCGA-A7-A26I-01, TCGA-AR-A1A...","{PRR20B, PRR20A, PRR20D, PRR20E, PRR20C}",{},"{21382, 21383, 21384, 21385, 21386}","{512, 128, 2, 1027, 132, 901, 387, 514, 260, 1...",UP,...,0.919120,0.974105,0.960947,NaN,NaN,NaN,NaN,0.880101,NaN,0.968112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,1.330680,219,364,"{AC005943.2, CTC-471J1.8, NADK2-AS1, RP11-529K...","{TCGA-AC-A3TM-01, TCGA-C8-A1HM-01, TCGA-E2-A14...",{},"{AC005943.2, CTC-471J1.8, NADK2-AS1, RP11-529K...","{3077, 22550, 8215, 8216, 25626, 13338, 2588, ...","{0, 1, 2, 3, 5, 7, 8, 11, 14, 16, 18, 28, 30, ...",DOWN,...,0.150813,0.378013,0.488284,NaN,NaN,NaN,NaN,0.896614,NaN,0.971126
141,1.314882,8,450,"{ZNF667, ZNF285, ZNF471, ZNF502, RP11-313J2.1,...","{TCGA-E2-A1L7-01, TCGA-AC-A3TM-01, TCGA-A8-A0A...",{},"{ZNF667, ZNF285, ZNF471, ZNF502, RP11-313J2.1,...","{38950, 38759, 38930, 38802, 25906, 39095, 390...","{0, 1, 6, 9, 13, 14, 15, 17, 23, 28, 30, 32, 3...",DOWN,...,0.941224,0.849122,0.960947,0.148856,0.770904,1.097541,0.541477,0.560535,0.213575,0.838812
142,1.308662,15,286,"{NAT16, CELSR3, RP11-1C8.4, OGDHL, BSN-AS2, TM...","{TCGA-JL-A3YW-01, TCGA-E2-A1L7-01, TCGA-E9-A22...","{NAT16, CELSR3, RP11-1C8.4, OGDHL, BSN-AS2, TM...",{},"{4163, 24740, 21029, 19078, 24743, 6632, 5607,...","{0, 1018, 515, 517, 519, 8, 9, 10, 11, 524, 10...",UP,...,0.596087,0.738905,0.789994,NaN,NaN,NaN,NaN,0.625537,NaN,0.882168
145,1.225185,4,470,"{ZNF626, CTD-2554C21.3, CTD-2554C21.2, ZNF90}","{TCGA-E2-A1L7-01, TCGA-AC-A3TM-01, TCGA-A8-A0A...",{},"{ZNF626, CTD-2554C21.3, CTD-2554C21.2, ZNF90}","{39276, 8092, 39067, 8091}","{0, 4, 5, 6, 11, 14, 17, 28, 31, 36, 38, 39, 4...",DOWN,...,0.257831,0.564697,0.587612,NaN,NaN,NaN,NaN,0.029833,NaN,0.238135


In [6]:
# adjusted p-value for OS, DFI or PFI is < 0.05:
biclusters[biclusters[["OS.p_value_BH","DFI.p_value_BH","PFI.p_value_BH"]].min(axis=1)<0.05].loc[:,["SNR","n_genes","n_samples","genes","direction",
                                                                                                    "OS.p_value_BH","DFI.p_value_BH","PFI.p_value_BH"]].sort_values(by="OS.p_value_BH")

,SNR,n_genes,n_samples,genes,direction,OS.p_value_BH,DFI.p_value_BH,PFI.p_value_BH
47,1.785266,327,364,"{TMEM26-AS1, RP11-12L8.1, KCTD6, KRT31, RP11-6...",DOWN,0.003893,0.234558,NaN
54,1.677363,100,164,"{RP11-261N11.8, LINC01208, LINC00458, RP11-734...",UP,0.033836,0.793449,0.104084
61,1.648707,267,420,"{PGK1, GSTA2, SMC1B, SPATA3, MTERF3, RP11-80H5...",UP,0.033836,0.455994,0.106898
50,1.761369,2,477,"{MSL3P1, AC005538.3}",DOWN,0.041547,0.550230,0.104084
118,1.447932,4,414,"{RP11-44F14.9, RP11-44F14.8, RP11-44F14.2, RP1...",DOWN,0.211663,0.174514,0.045597
120,1.435937,39,485,"{ATRNL1, RP1, RP11-284J1.1, RP11-328P23.3, RP1...",UP,0.249402,0.157037,0.047338


In [7]:
#" ".join(sorted(biclusters.loc[47,"genes"]))


### Associaiton with sex

In [8]:
# male samples
males = set(annot.loc[annot["sex"]==0,:].index.values) 
females = set(annot.loc[annot["sex"]==1,:].index.values) 
# all samples
N = annot.shape[0]  
print("%s males, %s females"%(len(males), len(females)))

12 males, 1086 females


In [9]:
from utils.eval import add_sex

add_sex(biclusters,males = males,females=females).sort_values(by=["sex.pval_BH"])

,SNR,n_genes,n_samples,genes,samples,genes_up,genes_down,gene_indexes,sample_indexes,direction,...,PFI.LogR_p_value,PFI.p_value_BH,PFI.LogR_p_value_BH,male.pval,male.counts,male.pval_BH,female.pval,female.counts,female.pval_BH,sex.pval_BH
119,1.445399,40,38,"{USP9Y, DAZ1, AC010084.1, TTTY10, XKRY, UTY, H...","{TCGA-EW-A1PD-01, TCGA-BH-A0B4-01, TCGA-BH-A0B...","{USP9Y, DAZ1, AC010084.1, TTTY10, XKRY, UTY, H...",{},"{39170, 37259, 11917, 23697, 786, 27157, 5655,...","{896, 387, 273, 658, 915, 767, 153, 923, 160, ...",UP,...,0.086039,NaN,0.338012,4.486112e-19,"(12, 26, 0, 1060)",4.934723e-17,1.000000,"(26, 12, 1060, 0)",1.0,4.934723e-17
31,2.075729,14,63,"{OR11K1P, OR4N4, OR4H6P, RP11-69H14.6, LINC011...","{TCGA-OL-A66P-01, TCGA-AC-A2QH-01, TCGA-A8-A09...","{OR11K1P, OR4N4, OR4H6P, RP11-69H14.6, LINC011...",{},"{19362, 19363, 15874, 19365, 30349, 30350, 184...","{387, 390, 777, 522, 141, 1039, 144, 786, 18, ...",UP,...,0.765519,NaN,0.929303,3.079526e-04,"(5, 58, 7, 1028)",1.693739e-02,0.999980,"(58, 5, 1028, 7)",1.0,1.693739e-02
102,1.497493,75,487,"{KRT14, SOX10, FAM189A2, CHST9, KRT17, AC01673...","{TCGA-E2-A1L7-01, TCGA-C8-A1HM-01, TCGA-A8-A0A...",{},"{KRT14, SOX10, FAM189A2, CHST9, KRT17, AC01673...","{27393, 35717, 27014, 6150, 17160, 39049, 9354...","{0, 4, 5, 6, 7, 8, 10, 13, 15, 23, 29, 30, 32,...",DOWN,...,0.447172,NaN,0.761532,8.808928e-04,"(11, 476, 1, 610)",2.422455e-02,0.999946,"(476, 11, 610, 1)",1.0,2.422455e-02
7,2.766905,5,75,"{PRR20B, PRR20A, PRR20D, PRR20E, PRR20C}","{TCGA-EW-A1PD-01, TCGA-A7-A26I-01, TCGA-AR-A1A...","{PRR20B, PRR20A, PRR20D, PRR20E, PRR20C}",{},"{21382, 21383, 21384, 21385, 21386}","{512, 128, 2, 1027, 132, 901, 387, 514, 260, 1...",UP,...,0.880101,NaN,0.968112,7.072819e-04,"(5, 70, 7, 1016)",2.422455e-02,0.999944,"(70, 5, 1016, 7)",1.0,2.422455e-02
101,1.505756,15,525,"{SERPING1, AL928768.3, IGLV7-43, IGLV8-61, CD7...","{TCGA-AO-A124-01, TCGA-E2-A14Y-01, TCGA-BH-A0C...",{},"{SERPING1, AL928768.3, IGLV7-43, IGLV8-61, CD7...","{5473, 13729, 13636, 36422, 13543, 13673, 6254...","{0, 5, 6, 7, 8, 10, 14, 15, 18, 19, 23, 25, 29...",DOWN,...,0.002045,NaN,0.191874,1.921151e-03,"(11, 514, 1, 572)",4.226533e-02,0.999866,"(514, 11, 572, 1)",1.0,4.226533e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50,1.761369,2,477,"{MSL3P1, AC005538.3}","{TCGA-AC-A3TM-01, TCGA-C8-A1HM-01, TCGA-AR-A24...",{},"{MSL3P1, AC005538.3}","{400, 17632}","{0, 3, 5, 6, 7, 11, 12, 17, 19, 21, 22, 27, 31...",DOWN,...,0.096570,0.104084,0.342555,6.568946e-01,"(5, 472, 7, 614)",1.000000e+00,0.571169,"(472, 5, 614, 7)",1.0,1.000000e+00
46,1.807128,46,51,"{SULT1C3, RP11-14K3.7, LINC01511, UGT2B24P, NO...","{TCGA-OL-A66P-01, TCGA-JL-A3YW-01, TCGA-OL-A5D...","{SULT1C3, RP11-14K3.7, LINC01511, UGT2B24P, NO...",{},"{4354, 29699, 10629, 28550, 37515, 5388, 37516...","{4, 265, 523, 1036, 141, 14, 12, 660, 532, 155...",UP,...,0.342008,NaN,0.671802,1.000000e+00,"(0, 51, 12, 1035)",1.000000e+00,0.563445,"(51, 0, 1035, 12)",1.0,1.000000e+00
43,1.842696,131,286,"{PRR33, C9orf173, CATSPER4, RP11-599B13.6, RP1...","{TCGA-OK-A5Q2-01, TCGA-JL-A3YW-01, TCGA-PL-A8L...","{PRR33, C9orf173, CATSPER4, RP11-599B13.6, RP1...",{},"{25091, 9219, 20998, 11270, 21011, 23062, 3432...","{513, 2, 515, 1028, 5, 3, 1027, 8, 1033, 1, 10...",UP,...,0.436100,NaN,0.761445,6.422413e-01,"(3, 283, 9, 803)",1.000000e+00,0.616343,"(283, 3, 803, 9)",1.0,1.000000e+00
54,1.677363,100,164,"{RP11-261N11.8, LINC01208, LINC00458, RP11-734...","{TCGA-JL-A3YW-01, TCGA-A8-A08X-01, TCGA-A2-A04...","{RP11-261N11.8, LINC01208, LINC00458, RP11-734...",{},"{28160, 15884, 26637, 30738, 11810, 6180, 6181...","{513, 4, 6, 520, 1034, 1035, 12, 1036, 11, 10,...",UP,...,0.033116,0.104084,0.238135,8.579911e-01,"(1, 163, 11, 923)",1.000000e+00,0.444797,"(163, 1, 923, 11)",1.0,1.000000e+00
